In [1]:
!pip install ../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl
!pip install ../input/timm-pytorch-image-models/pytorch-image-models-master
!pip install ../input/hpacellsegmentatormaster/HPA-Cell-Segmentation-master
!pip install ../input/hpapytorchzoozip/pytorch_zoo-master

Processing /kaggle/input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl
Processing /kaggle/input/timm-pytorch-image-models/pytorch-image-models-master
  Created wheel for timm: filename=timm-0.4.8-py3-none-any.whl size=325934 sha256=811cd284254ec023a6e261418fdde6251959c972c8cbb231a5301672afe16c0f
  Stored in directory: /root/.cache/pip/wheels/b2/4e/24/ca2e6fc7fceb1e8f1f4d3e5dd21df64327a03cf318d915c1bb
Successfully built timm
Processing /kaggle/input/hpacellsegmentatormaster/HPA-Cell-Segmentation-master
  Created wheel for hpacellseg: filename=hpacellseg-0.1.7-py3-none-any.whl size=14777 sha256=f3c42b8c0c9343212b883ffe19a8308ac05079f11d520247b900b25bca59a6ab
  Stored in directory: /root/.cache/pip/wheels/9d/fe/fb/7155f898e42718e2bf20deda36e64aae6b19b5714f55110899
Successfully built hpacellseg
Processing /kaggle/input/hpapytorchzoozip/pytorch_zoo-master
  Created wheel for pytorch-zoo: filename=pytorch_zoo-0.0.0-py3-none-any.whl size=30139 sha256=488ebbb48819b4d955565987f3101d5a

In [2]:
import pandas as pd
import numpy as np
import os
import cv2
from tqdm import tqdm
import shutil
import matplotlib.pyplot as plt
import base64
from pycocotools import _mask as coco_mask
import typing as t
import zlib
import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei
from fastai.vision.all import *
from timm import create_model
from fastai.vision.learner import _update_first_layer
from torch.distributions.beta import Beta
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
train_df = pd.read_csv('../input/hpa-single-cell-image-classification/train.csv')
train_df['fname'] = train_df['ID'].apply(lambda x: '../input/hpa-single-cell-image-classification/train/' + x + '_green.png')
train_df = train_df.rename(columns = {'Label': 'labels'})

In [4]:
# sample csv
sub_df = pd.read_csv('../input/hpa-single-cell-image-classification/sample_submission.csv')
sub_df = sub_df.rename(columns = {'ID': 'Image'})

# df with per cell encs
out_df = pd.DataFrame(columns = ['img_id', 'fname', 'enc'])

In [5]:
# cell extractor details
save_dir = '/kaggle/working/test_cells'
img_path = '../input/hpa-single-cell-image-classification/test'

if not os.path.exists(save_dir):
    os.mkdir(save_dir)

In [6]:
def encode_binary_mask(mask: np.ndarray) -> t.Text:
    if mask.dtype != np.bool:
        raise ValueError(
        "encode_binary_mask expects a binary mask, received dtype == %s" %
        mask.dtype)

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError(
        "encode_binary_mask expects a 2d mask, received shape == %s" %
        mask.shape)

    # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

      # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

      # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str.decode('ascii')

In [7]:
def get_cells(img_name, cell_mask, out_df):
    num_cells = np.max(cell_mask)

    r = cv2.imread('{}/{}_red.png'.format(img_path, img_name), cv2.IMREAD_UNCHANGED)
    g = cv2.imread('{}/{}_green.png'.format(img_path, img_name), cv2.IMREAD_UNCHANGED)    
    b = cv2.imread('{}/{}_blue.png'.format(img_path, img_name), cv2.IMREAD_UNCHANGED)
    
    for cell in range(1, num_cells + 1):
        b_mask = (cell_mask == cell)
        true_points = np.argwhere(b_mask)
        top_left = true_points.min(axis=0)
        bottom_right = true_points.max(axis=0)
        
        r_m = r * b_mask
        g_m = g * b_mask
        b_m = b * b_mask
        
        r_m = r_m[top_left[0]:bottom_right[0]+1,top_left[1]:bottom_right[1]+1]  
        g_m = g_m[top_left[0]:bottom_right[0]+1,top_left[1]:bottom_right[1]+1]        
        b_m = b_m[top_left[0]:bottom_right[0]+1,top_left[1]:bottom_right[1]+1]
        
        
        
        if r_m.max() > 255:
            im = np.dstack((b_m, g_m, r_m)) / 255
        else:
            im = np.dstack((b_m, g_m, r_m))
        
        fname = '{}/{}_{}.png'.format(save_dir, img_name, cell)
        enc = encode_binary_mask(b_mask)
        
        out_df = out_df.append({'img_id': img_name, 'fname': fname, 'enc': enc}, ignore_index = True)
        
        cv2.imwrite(fname, im)   
    
    return out_df

In [8]:
colors = ['red', 'yellow', 'blue']
NUC_MODEL = "../input/hpacellsegmentatormodelweights/dpn_unet_nuclei_v1.pth"
CELL_MODEL = "../input/hpacellsegmentatormodelweights/dpn_unet_cell_3ch_v1.pth"

segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    scale_factor= 0.25,
    device="cuda",
    padding=True,
    multi_channel_model=True)

please compile abn


/opt/conda/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'pytorch_zoo.unet.DPNUnet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set

In [9]:
out_path = '/kaggle/working/test_rgb'
raw_img_path = '../input/hpa-single-cell-image-classification/test'

if not os.path.exists(out_path):
    os.mkdir(out_path)

for idx, row in tqdm(sub_df.iterrows(), total = len(sub_df)):
    img_name = row.Image
    
    b = cv2.imread('{}/{}_blue.png'.format(raw_img_path, img_name), cv2.IMREAD_UNCHANGED)
    g = cv2.imread('{}/{}_green.png'.format(raw_img_path, img_name), cv2.IMREAD_UNCHANGED)
    r = cv2.imread('{}/{}_red.png'.format(raw_img_path, img_name), cv2.IMREAD_UNCHANGED)
    
    if b.max() > 255: b = b / 255
    if g.max() > 255: g = g / 255
    if r.max() > 255: r = r / 255
    
    im = np.dstack((b, g, r))
    im = cv2.resize(im, (448, 448))
    
    cv2.imwrite('{}/{}.png'.format(out_path, img_name), im)

100%|██████████| 559/559 [04:03<00:00,  2.30it/s]


In [10]:
for idx, row in tqdm(sub_df.iterrows(), total = len(sub_df)):
    img_name = row.Image.split('/')[-1]
    channels = np.expand_dims(np.array(['{}/{}_{}.png'.format(img_path, img_name, color) for color in colors]), axis = 1).tolist()
    
    nuc_segmentations = segmentator.pred_nuclei(channels[2])
    cell_segmentations = segmentator.pred_cells(channels)
      
    nuclei_mask, cell_mask = label_cell(nuc_segmentations[0], cell_segmentations[0])
        
    out_df = get_cells(img_name, cell_mask, out_df)

100%|██████████| 559/559 [1:37:27<00:00, 10.46s/it]


In [11]:
out_df.head()

,img_id,fname,enc
0,0040581b-f1f2-4fbe-b043-b6bfea5404bb,/kaggle/working/test_cells/0040581b-f1f2-4fbe-b043-b6bfea5404bb_1.png,eNqtVm2PojAQ/ksMZe+Sy21yubtc4gKDIkpBUJQ3KyD//9t1KPhu1M22Rp9Op0+n0+mM87cMoj1osoHmgDN8aeCormcu7nBXgQ+ehxXWFczA9bDBPaF6ihPHrWECyRR/hTV4ulyuSLqm0GFMxOpDvRP18Ep+tynLSBV1RzGc8BzGg16HSIq67ArBrfnL9Zp2i/y2fHCXkg/EEndH0wZn4nGL83Un/cxvL5z/znmGbR/547j9q357wNPbeTj+eVfOUeZqzsl659RF2iFSDuMhWAaL5B79fB9lpOKPFk7p41Zbv9liAz9nMf6tN/Duxvin3UCiNRH+8lIotTxC7kQpCC2NcO4sU6i0dYSek6TQanGEYyeTMn0domcnOTT6MsSpneaw16MQJ3aeg8t4iI0lcghZG2BhuQUsWB1gbs0KiJgIMLN4ASuWB5hYUQEFiwMMrayAkoUBBlZZwJb5AXJrV4BgXoC+1RRQsUmAnjUuoWEtR9dyS2hZw3FseSVMDcGxNnkJcyPnKMy4BG5sOBZmUsLCWHHMzE0JsRFxTM28hKURcNyY2xJWxpzj2qxKSIwpoYbQmGbHW6nX+hK5W6lX...
1,0040581b-f1f2-4fbe-b043-b6bfea5404bb,/kaggle/working/test_cells/0040581b-f1f2-4fbe-b043-b6bfea5404bb_2.png,eNrVUm1PwkAM/kvt3XgRjEajvO7uhGCEEHCgcSQi/v9vXtudbGwzQT+5JteXe67tnna7fE1wC6tPVA5tFGujrDLa+tOhjyixRFcI421kPMKBy6KmEbeG9F7H7QG91+YJd1YqRLEgLu8Ewb5HdGv8kCGFBXXkrdZQ9AGmHEFfVyIXfXQAmEW1aQ/yPqHgd59DOb2AHORK5nocQADn4siaOwZkTrL+hPGAEs6DXRL5Y+G7dCvvaIolHJ+W5xtrV5it0TJ3b0U01bg5Zn2csmxHzudZ2WisbaFeyN/rFPu5uSY2rOp1iBej+h2ZBxMJhGLGgB3yX+JVaqfPbrLAfQKPqT2s3cccpxuy92v3PsfZBp5Su127xMBiibsE3nx1ynp7xRrvu0ZL/Wo9bFXHA/vEzDn+t2TTpxuyLJ7lQ3GzQPLxFvl7xzuV9y0S3qpRo9TfsTNCjxt/+7/a/62R/5I/xHmPf+Br1KzOd7pPp/iwh6f7lsUnD/sZfgE+j18v
2,0040581b-f1f2-4fbe-b043-b6bfea5404bb,/kaggle/working/test_cells/0040581b-f1f2-4fbe-b043-b6bfea5404bb_3.png,eNoLCEgJMg4wDMg3NPQ39Df2NfaF0EZ+IBoBIXzc8oSgARxA2OiiuMWJBYT0kyuPLk4aH+F/6ojTS57a5tALwtIpqemTVvYDaQMDIwM/eJoCiQJJXO4EhqCBPyQ8/Yz8DbAlLScA3+jcjQ==
3,0040581b-f1f2-4fbe-b043-b6bfea5404bb,/kaggle/working/test_cells/0040581b-f1f2-4fbe-b043-b6bfea5404bb_4.png,eNptUstOAzEM/CUnXtoiAQcegnbzOMAFgdRFbIXEpf9/q+2pd0vYjZQde8aPxBmHz4EG2h9DLDGtHg5hLIJK198+muf+i7YKusQ11JD5vf5QjUXQ041KE+9WagsbLVRRu1re9sKI769qIHKt+hyhDnRzXvCZMy/zrU76W/R7Ha1N5B3Mnfii6RMsDgHqrbrDINWc7SnExJf6i0RzsaY42iNkXdChJbXswK3er3jiEedHEZXzgv7ni/P1GAq5U32O55FFjAeKIgPowXPBcDmjfoSdO/D2eOw17dTmhIGx+l0v/Qj7l3+5hv28Sfpf7+sdHB91JHO9fb/+xhOdSqvy
4,0040581b-f1f2-4fbe-b043-b6bfea5404bb,/kaggle/working/test_cells/0040581b-f1f2-4fbe-b043-b6bfea5404bb_5.png,eNp9UcuSmzAQ/CVGku1U8CGpxBs7EsMrPNdgMNYasIH/v0UD61RctbXo0D2tHqk1/Okvr5Dw9g4MmRL7Lzjjt8payE/bZQgO323nmu9sF1xTdy+ZOmvMRZFDK2uNmShz6GSlMSVtlIXGSNQ5eCrTGIq3HCaZagzElbREoy/aHEIVaPTEPYdC5Q0GK30ET2YN+qu3I/gybdAjLZZRg4O4HGGQZYXluj5BsKJkv74iM5ksyuWC9cEH7meLIXcI7Z1B80ihBNVYWd+XDe4smECHz5btewt3KAFgY5Xu4t0cFjSzc8nBcHHYL89nGp9ijxMM2rvlZOY+wtMN5LxYBb2y2F8TLDd5C/V+SDDbVC08ZeLvN3Yxng5BB6f1FGNNrFyPMZ4PIbEbsaSD85rjc6IHzsk/GJNJSjuFm0Q4yL4HX6QRjrIjZrSJmEeapwwbuWGhuvdw5VGEqZp60KRNciRfSL0TsYA6vJvp8CIMlH+DgZ/nv4rM4csMaYrmhf9mhv8v7ohZ3/5Y/RZUq6XLi8OI/QVl/s76


In [12]:
def get_image_path(x): return x.fname

single_labels = [str(i) for i in range(19)]

In [13]:
item_tfms = Resize(448, method = 'squish')
batch_tfms = [Rotate(max_deg = 270, p = 0.9), Normalize()]
bs = 32

In [14]:
dblock = DataBlock(blocks = (ImageBlock, MultiCategoryBlock(vocab = single_labels)),
                splitter = RandomSplitter(),
                get_x = get_image_path,
                get_y = ColReader('labels', label_delim = '|'),
                item_tfms = item_tfms,
                batch_tfms = batch_tfms)

dls = dblock.dataloaders(train_df, bs = bs)

In [15]:
def create_timm_body(arch:str, pretrained=False, cut=None, n_in=3):
    "Creates a body from any model in the `timm` library."
    model = create_model(arch, pretrained=pretrained, num_classes=0, global_pool='')
    _update_first_layer(model, n_in, pretrained)
    if cut is None:
        ll = list(enumerate(model.children()))
        cut = next(i for i,o in reversed(ll) if has_pool_type(o))
    if isinstance(cut, int): return nn.Sequential(*list(model.children())[:cut])
    elif callable(cut): return cut(model)
    else: raise NamedError("cut must be either integer or function")

def create_timm_model(arch:str, n_out, cut=None, pretrained=False, n_in=3, init=nn.init.kaiming_normal_, custom_head=None,
                     concat_pool=True, **kwargs):
    "Create custom architecture using `arch`, `n_in` and `n_out` from the `timm` library"
    body = create_timm_body(arch, pretrained, None, n_in)
    if custom_head is None:
        nf = num_features_model(nn.Sequential(*body.children()))
        head = create_head(nf, n_out, concat_pool=concat_pool, **kwargs)
    else: head = custom_head
    model = nn.Sequential(body, head)
    if init is not None: apply_init(model[1], init)
    return model

def timm_learner(dls, arch:str, loss_func=None, pretrained=False, cut=None, splitter=None,
                y_range=None, config=None, n_out=None, normalize=True, **kwargs):
    "Build a convnet style learner from `dls` and `arch` using the `timm` library"
    if config is None: config = {}
    if n_out is None: n_out = get_c(dls)
    assert n_out, "`n_out` is not defined, and could not be inferred from data, set `dls.c` or pass `n_out`"
    if y_range is None and 'y_range' in config: y_range = config.pop('y_range')
    model = create_timm_model(arch, n_out, default_split, pretrained, y_range=y_range, **config)
    learn = Learner(dls, model, loss_func=loss_func, splitter=default_split, **kwargs)
    if pretrained: learn.freeze()
    return learn

In [16]:
# First model

model_name1 = 'inception_v3'
weights1 = '/kaggle/input/submission-data/inv3_gc_resize448r_8epcs_32bs_0.468'
learn1 = timm_learner(dls = dls, arch = model_name1, metrics = [accuracy_multi, PrecisionMulti()]).to_fp16()
learn1 = learn1.load(weights1)
tta = 4

/opt/conda/lib/python3.7/site-packages/fastai/learner.py:56: UserWarning: Saved filed doesn't contain an optimizer state.
  elif with_opt: warn("Saved filed doesn't contain an optimizer state.")


In [17]:
test_dl1 = learn1.dls.test_dl(out_df)
preds1, _ = learn1.tta(dl = test_dl1, n = tta)

In [18]:
print('Done model 1')

Done model 1


In [19]:
# Second model

model_name2 = 'mobilenetv3_large_100'
weights2 = '/kaggle/input/submission-data/mbl100_floss'
learn2 = timm_learner(dls = dls, arch = model_name2, metrics = [accuracy_multi, PrecisionMulti()]).to_fp16()
learn2 = learn2.load(weights2)
tta = 4

In [20]:
test_dl2 = learn2.dls.test_dl(out_df)
preds2, _ = learn2.tta(dl = test_dl2, n = tta)

In [21]:
print('Done model 2')

Done model 2


In [22]:
preds_c = ((preds1 + preds2) / 2).numpy()

In [23]:
# Third model - image level

model_name3 = 'efficientnet_b3'
weights3 = '/kaggle/input/submission-data/efficientnet_b3'
learn3 = timm_learner(dls = dls, arch = model_name3, metrics = [accuracy_multi, PrecisionMulti()]).to_fp16()
learn3 = learn3.load(weights3)
tta = 4

In [24]:
sub_df['fname'] = sub_df['Image'].apply(lambda x: out_path + '/' + x + '.png')

In [25]:
test_dl3 = learn3.dls.test_dl(sub_df)
preds3, _ = learn3.tta(dl = test_dl3, n = tta)

In [26]:
sa = pd.read_csv('../input/hpa-single-cell-image-classification/sample_submission.csv')
slide_preds = []

for idx, row in tqdm(out_df.iterrows(), total = len(out_df)):
    test_idx = sa[sa['ID'] == row.img_id].index.values[0]
    slide_preds.append(preds3[test_idx].numpy())

100%|██████████| 10716/10716 [00:06<00:00, 1590.26it/s]


In [27]:
preds = (preds_c + slide_preds) / 2

In [28]:
out_df['pred_enc'] = np.nan
out_df = out_df.reset_index(drop = True)

In [29]:
for idx, row in tqdm(out_df.iterrows(), total = len(out_df)):
    pred_enc_string = ""
    
    for p_idx, pred in enumerate(preds[idx]):
        pred_enc_string += str(p_idx) + " " + str(float(preds[idx][p_idx])) + " " + row.enc + " "
    
    out_df.iloc[idx, len(out_df.columns) - 1] = pred_enc_string

100%|██████████| 10716/10716 [00:05<00:00, 1866.60it/s]


In [30]:
sample_df = pd.read_csv('../input/hpa-single-cell-image-classification/sample_submission.csv')
sample_df = sample_df[['ID', 'ImageWidth', 'ImageHeight']]
sample_df['PredictionString'] = np.nan
sample_df.head()

,ID,ImageWidth,ImageHeight,PredictionString
0,0040581b-f1f2-4fbe-b043-b6bfea5404bb,2048,2048,NaN
1,004a270d-34a2-4d60-bbe4-365fca868193,2048,2048,NaN
2,00537262-883c-4b37-a3a1-a4931b6faea5,2048,2048,NaN
3,00c9a1c9-2f06-476f-8b0d-6d01032874a2,2048,2048,NaN
4,0173029a-161d-40ef-af28-2342915b22fb,3072,3072,NaN


In [31]:
for idx, row in tqdm(sample_df.iterrows(), total = len(sample_df)):
    sub_df = out_df[out_df['img_id'] == row.ID]
    enc_strs = sub_df.pred_enc.values
    sample_df.iloc[idx, 3] = " ".join(enc_strs)

100%|██████████| 559/559 [00:01<00:00, 373.69it/s]


In [32]:
print('Done making df')

Done making df


In [33]:
sample_df

,ID,ImageWidth,ImageHeight,PredictionString
0,0040581b-f1f2-4fbe-b043-b6bfea5404bb,2048,2048,0 0.28665125370025635 eNqtVm2PojAQ/ksMZe+Sy21yubtc4gKDIkpBUJQ3KyD//9t1KPhu1M22Rp9Op0+n0+mM87cMoj1osoHmgDN8aeCormcu7nBXgQ+ehxXWFczA9bDBPaF6ihPHrWECyRR/hTV4ulyuSLqm0GFMxOpDvRP18Ep+tynLSBV1RzGc8BzGg16HSIq67ArBrfnL9Zp2i/y2fHCXkg/EEndH0wZn4nGL83Un/cxvL5z/znmGbR/547j9q357wNPbeTj+eVfOUeZqzsl659RF2iFSDuMhWAaL5B79fB9lpOKPFk7p41Zbv9liAz9nMf6tN/Duxvin3UCiNRH+8lIotTxC7kQpCC2NcO4sU6i0dYSek6TQanGEYyeTMn0domcnOTT6MsSpneaw16MQJ3aeg8t4iI0lcghZG2BhuQUsWB1gbs0KiJgIMLN4ASuWB5hYUQEFiwMMrayAkoUBBlZZwJb5AXJrV4BgXoC+1RRQsUmAnjUuoWEtR9dyS2hZw3FseSVMDcGxNnkJcyPnKMy4BG5sOBZmUsLCWHHMzE0JsRFxTM28hKURcNyY2xJWxpzj2qxKSI...
1,004a270d-34a2-4d60-bbe4-365fca868193,2048,2048,0 0.07058369368314743 eNoLCEhJMAgwDMg3NPQ39Df2tXaF0PaOENrYF0Kbe6LyrdxAtKG/kR+Eb+aFyoeZA1OPLg+hcUMDAxAGkRAeJp9yALEKwsInjspHdyoheWLV43IPbnnS7MUO/Yz8jJBt8Db1NYaIE9IHUedqjareywzC9zSH0MAEAKIDA2JyjADDeJGw 1 0.03284187614917755 eNoLCEhJMAgwDMg3NPQ39Df2tXaF0PaOENrYF0Kbe6LyrdxAtKG/kR+Eb+aFyoeZA1OPLg+hcUMDAxAGkRAeJp9yALEKwsInjspHdyoheWLV43IPbnnS7MUO/Yz8jJBt8Db1NYaIE9IHUedqjareywzC9zSH0MAEAKIDA2JyjADDeJGw 2 0.056898247450590134 eNoLCEhJMAgwDMg3NPQ39Df2tXaF0PaOENrYF0Kbe6LyrdxAtKG/kR+Eb+aFyoeZA1OPLg+hcUMDAxAGkRAeJp9yALEKwsInjspHdyoheWLV43IPbnnS7MUO/Yz8jJBt8Db1NYaIE9IHUedqjareywzC9zSH0MAEAKIDA2JyjADDeJG...
2,00537262-883c-4b37-a3a1-a4931b6faea5,2048,2048,0 0.04897116497159004 eNqFUE0LwjAM/UtJuqlM8AsUN9fWgzcVlDEPMvD/32yS1m2IuLC8vOSlafM4dlC0LySP3tQn7Bw58sbOt8gZu1wrTkvFBq5eozO2TqOgBYDQZwMH+miNVSTRoQ8ourxiHXhyX/UR/2uOxMeTg2dDZxjrrM4S/ugH4J99Yj+Nr13syIlchoWh0gRcFCfB8NNkSie95D2PxsiGYwb68UUxTuPlUP/4tAh9elqB18i4WOETmB9kJdZEJa+670zdoS6rC94mbrhW59q/nwnPy0klfHUDSUxWC0lkG8X7s7mYNwcdlKw= 1 0.013224027119576931 eNqFUE0LwjAM/UtJuqlM8AsUN9fWgzcVlDEPMvD/32yS1m2IuLC8vOSlafM4dlC0LySP3tQn7Bw58sbOt8gZu1wrTkvFBq5eozO2TqOgBYDQZwMH+miNVSTRoQ8ourxiHXhyX/UR/2uOxMeTg2dDZxjrrM4S/ugH4J99Yj+Nr13syIlchoWh0gRcFCfB8NNkSie95D2PxsiGYwb68UUxTuPlUP/4tAh9elqB18i4WOETmB9kJdZEJa...
3,00c9a1c9-2f06-476f-8b0d-6d01032874a2,2048,2048,0 0.03800433129072189 eNrtVUtvwjAM/ktxAutp2mGXDhpHGpDCxvuZQtn+/21OnZQCEkyaNO2wWIqftROr/lL0bQ+eFp+gNKAkAiEM7YYIpd+1Yn4iiuAoifyV4vjKwlalH59bXf+dzJTfOR95SEIIFap65KvyiWA/q0+SbnXadT3SqRpHhZp0vocO5YiW2hNJAOev/eJal6bdBX8b1uNqxPlzK02nPIuICbSK5opf5r9c9/z3luH7Vh0xtwLB8F71Ia5Zjtt07OAtWf9M2qXvDkbJJsePtO/gNSl/WzqmAwe95JBjmVoH/WT/L/1JaZA4Lw0dTJPv/+ZhhLUUIJpzfRq3xjiBuRwoBq7bFW44PYydA4qPbwxSc47F2dyHvCjD8NXBgK1MYoBDE0c0wmdVL8LfLTohT5Su9bqkPzajYNCNYGgts4nFzqYAubDYnReglhazGekri3pagNlZFKMCsoNFZQvolhZbgwI6R+bpMEcsHOAkR7N2YKY5ihXpM+Z6kSPMiS9zlDMH2arBxyW8cDPChQVhvW+AfyWqFvH7sHXHPXwBvVrJXQ== 1 0.00746390642598271...
4,0173029a-161d-40ef-af28-2342915b22fb,3072,3072,0 0.2537965476512909 eNqtlFuXmjAQgP8SDOgB2peeLsueXccgbgWRiwKRYBf2/781AYOKaN2eJmq+uSSZTCYqq51ySEDhTSXdr+hyvN1v+Q/117JsX9U/an+oVRlGMfFiKDKlkUwz5aPl9xiqTCGTN+ke/EgL3GE+p2YTQflCC0y4xEwWQfVSFhhzqTL3QqoKjLj028wj2DtViWvDD6Bw6hI3xlJQU2JkeAHkjss4uQHsnCXD1Kh9SJwVw8w4+BA7gSAmaM0wN0ofVs6W4cHY+rB0coa1EfvgCfoU5DoFQ9dcpaAQReVht/khIC9N5o7r1C/mq5sglzndxKh+xJ904cx0Me1f97tuczg5jJ1tFm/w8FQU8OubPl5LfQ2pP9cbTG1WQGqFG9zapSCu2/UU2qqyL2AveG1rChUcCP0tXlCg1meIG9ujUFhNiJG9pJBbdYjxKL1TyKyPEJMhTQcPavgcYH7+OIde44cPQqztPQXXCkN0n0sKn+Y6xMUzo9CY3OqdkabEgrf3OaFQm2mIy2tuzFjwhHD2rCzExr5PWwoLKxGR5RRW1ums8uL4VyUaPlLA3G9+nRvUumr...
...,...,...,...,...
554,fea47298-266a-4cf4-93bd-55d1bcc2fc7d,1728,1728,0 0.10701867192983627 eNqFUNsOgjAM/aV1XMQ/gOHa4P1BECTGS4L+/5uMrhEv0TZZu3NO262rw6nUG31uQaPGMJ+mfZzF+VHtiJHQOiQ0MWoCAgQauQ0Y9a6U6g/oAym5eXPZSMKRM9LkdaBcJBD90MjpFMNSD0N/ac59PF/u8W62NRTRxWXrGubRdY83s6ph6bDObGqoIv/iNOH6D+d/2VkEv/lQq/EeDOttkE14P1mCQyyp5dcS5MNwWUzeNnhKywrWk+SJfjP5r1O5cTzylRPr2UWDRdZVcIsDQW3wOkHu7/h//gE0yobA 1 0.036053698509931564 eNqFUNsOgjAM/aV1XMQ/gOHa4P1BECTGS4L+/5uMrhEv0TZZu3NO262rw6nUG31uQaPGMJ+mfZzF+VHtiJHQOiQ0MWoCAgQauQ0Y9a6U6g/oAym5eXPZSMKRM9LkdaBcJBD90MjpFMNSD0N/ac59PF/u8W62NRTRxWXrGubRdY83s6ph6bDObGqoIv/iNOH6D+d/2VkEv/lQq/EeDOttkE14P1mCQyyp5dcS5MNwWUzeNnhKywrWk+SJfjP5r1...
555,feb955db-6c07-4717-a98b-92236c8e01d8

In [34]:
shutil.rmtree(save_dir)
shutil.rmtree(out_path)
shutil.rmtree('/kaggle/working/models')

In [35]:
sample_df.to_csv('submission.csv', index=False)